In [2]:
rm -rf /kaggle/working/*

In [3]:
!git clone --branch main https://github.com/giankev/Ancient-to-Modern-Italian-Automatic-Translation.git

Cloning into 'Ancient-to-Modern-Italian-Automatic-Translation'...
remote: Enumerating objects: 132, done.
remote: Counting objects: 100% (132/132), done.
remote: Compressing objects: 100% (112/112), done.
remote: Total 132 (delta 69), reused 50 (delta 16), pack-reused 0 (from 0)
Receiving objects: 100% (132/132), 390.65 KiB | 5.66 MiB/s, done.
Resolving deltas: 100% (69/69), done.


In [4]:
import torch

if torch.cuda.is_available():
    device = torch.device("cuda:0")
    print(f"Using GPU: {torch.cuda.get_device_name(0)}")
else:
    device = torch.device("cpu")
    print("CUDA (GPU) not available, using CPU.")

Using GPU: Tesla T4


In [5]:
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0)) 
print(torch.cuda.get_device_name(1)) 

2
Tesla T4
Tesla T4


In [6]:
import pandas as pd

gemma2b_it_cl_translation = pd.read_csv("/kaggle/working/Ancient-to-Modern-Italian-Automatic-Translation/Gemma2b-it-translation/CulturalI-hw2_transl-Gemma2b-it_context_learning.csv")
dataset_gold = pd.read_csv("/kaggle/working/Ancient-to-Modern-Italian-Automatic-Translation/dataset/dataset_gold.csv")

In [ ]:
gemma2b_it_cl_translation.head()

In [ ]:
dataset_gold.head()

In [7]:
df_concat = pd.concat([gemma2b_it_cl_translation, dataset_gold], axis=1)  # axis=0 indica concatenazione per righe (verticale)

In [ ]:
df_concat.head()

In [8]:
df_concat.rename(columns={'translation': 'response'}, inplace=True)
df_concat.rename(columns={'Translation': 'reference_answer'}, inplace=True)

In [9]:
df_concat = df_concat[["response","reference_answer"]]
df_concat.head()

,response,reference_answer
0,"""Quella guerra ben fatta, l'opera perché etc. ...",Quella guerra è stata combattuta per una giust...
1,"""crudele, e di tutte le colpe si prende vendet...",È crudele e punisce ogni colpa come vuole la l...
2,"""Non ha altra forza d'animo che l'onore di Pon...","Ponzio Aufidiano non aveva coraggio d’animo, e..."
3,"""Se questo piace a tutti e se il tempo ha biso...",Se per tutti va bene avere Pompeo come cavalie...
4,"""L'arte di questa disciplina sembra essere que...",Il compito di quest’arte sembra essere parlare...


In [10]:
ABS_SYSTEM_PROMPT = """You are a fair judge assistant tasked with providing clear,
                     objective feedback based on specific criteria, ensuring each assessment
                     reflects the absolute standards set for performance."""

ABSOLUTE_PROMPT = """###Task Description:
An instruction (might include an Input inside it), a response to evaluate, a reference answer that gets a score of 5, and a score rubric representing a evaluation criteria are given.
1. Write a detailed feedback that assess the quality of the response strictly based on the given score rubric, not evaluating in general.
2. After writing a feedback, write a score that is an integer between 1 and 5. You should refer to the score rubric.
3. The output format should look as follows: "Feedback: (write a feedback for criteria) [RESULT] (an integer number between 1 and 5)"
4. Please do not generate any other opening, closing, and explanations.

###The instruction to evaluate:
{instruction}

###Response to evaluate:
{response}

###Reference Answer:
{reference_answer}

###Score Rubrics:
{rubric}

###Feedback: """

def build_prometheus_prompt(response_given: str, reference_answer: str):
    instruction = "Evaluate the translation from Old Italian to Modern Italian."
    response = response_given
    reference = reference_answer
    rubric = """Semantic fidelity and grammatical correctness:
                Score 1: Completely unacceptable translation: irrelevant, incomprehensible, or nonsensical.
                Score 2: Severe semantic errors, substantial omissions or additions. Defective syntax.
                Score 3: Mediocre translation with minor semantic mistakes or typos.
                Score 4: Good translation, faithful and understandable with slight stylistic imperfections.
                Score 5: Perfect translation: accurate, fluent, coherent, and semantically correct."""

    return ABS_SYSTEM_PROMPT + "\n\n" + ABSOLUTE_PROMPT.format(
        instruction=instruction,
        response=response_given,
        reference_answer=reference_answer,
        rubric=rubric
    )

In [11]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained("prometheus-eval/prometheus-7b-v2.0", device_map = "auto")
tokenizer = AutoTokenizer.from_pretrained("prometheus-eval/prometheus-7b-v2.0")

2025-06-08 15:56:21.465973: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749398181.489253     259 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749398181.496211     259 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [47]:
from tqdm import tqdm
import os

In [46]:
def evaluations(df, output_csv_path):
    
    results = []
    for index, item in tqdm(df.iterrows(), total=df.shape[0], desc="Evaluations", unit="phrase"):
        response = item["response"]
        reference_answer = item["reference_answer"]
        prompt_text = build_prometheus_prompt(response, reference_answer)
        
        messages = [
            {"role": "user", "content": prompt_text},
        ]
        
        encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt")
        
        
        if tokenizer.pad_token_id is None:
           tokenizer.pad_token_id = tokenizer.eos_token_id

        #encodeds = {k: v.to(model.device) for k, v in encodeds.items()}

        with torch.no_grad():
            generated_ids = model.generate(
                            encodeds,
                            max_new_tokens=1000,
                            do_sample=False,   
                            temperature=0.0,    
                            pad_token_id=tokenizer.pad_token_id, 
                            eos_token_id=tokenizer.eos_token_id     
                        )
            
        decoded = tokenizer.batch_decode(generated_ids)
    
        score = ""
        if "[RESULT]" in decoded[0]:
          score = decoded[0].split("[RESULT]")[-1].strip().replace("</s>", "").strip() 
    
        if (index + 1) % 10 == 0:
                print(f"\nProcessed {index + 1} phrase.")
    
        results.append({
                'response': response,
                'reference_answer': reference_answer,
                'score': score
            })
        
    df_output = pd.DataFrame(results)
    os.makedirs(os.path.dirname(output_csv_path), exist_ok=True)
    df_output.to_csv(output_csv_path, index=False, encoding='utf-8')
        
    print(f"\nTranslation complete. Results saved in: {output_csv_path}")

In [48]:
output_csv_path = "/kaggle/working/Gemma2b-it-evaluations_prometheus.csv"
evaluations(df_concat, output_csv_path)

Evaluations:  10%|█         | 10/97 [04:18<37:20, 25.75s/phrase]


Processed 10 phrase.


Evaluations:  21%|██        | 20/97 [08:44<34:57, 27.24s/phrase]


Processed 20 phrase.


Evaluations:  31%|███       | 30/97 [13:07<30:41, 27.49s/phrase]

...


Processed 80 phrase.


Evaluations:  93%|█████████▎| 90/97 [42:15<02:39, 22.85s/phrase]


Processed 90 phrase.


Evaluations: 100%|██████████| 97/97 [45:31<00:00, 28.16s/phrase]


Translation complete. Results saved in: /kaggle/working/Gemma2b-it-evaluations_prometheus.csv


In [49]:
import pandas as pd

# Carica il CSV
df = pd.read_csv("/kaggle/working/Gemma2b-it-evaluations_prometheus.csv")

# Salva in formato JSON Lines
df.to_json("/kaggle/working/Gemma2b-it-evaluations_prometheus.jsonl", orient="records", lines=True, force_ascii=False)